In [1]:
import pandas as pd
import numpy as np
import openpyxl
import functools

In [2]:
# Задание 1
# Для датафрейма log из материалов занятия создайте столбец source_type по правилам:

# если источник traffic_source равен Yandex или Google, то в source_type ставится organic;
# для источников paid и email из России ставим ad;
# для источников paid и email не из России ставим other;
# все остальные варианты берём из traffic_source без изменений.

log = pd.read_csv('./hw_4/visit_log.csv', sep=';') # загружаю данные

def source_type(row): # функция, которая будет ставить в столбец нужные значения
    out = row['traffic_source']
    if row['traffic_source'].lower() == 'yandex' or row['traffic_source'].lower() == 'google':
        out = 'organic'
    if (row['traffic_source'].lower() == 'paid' or row['traffic_source'].lower() == 'email'):
        if row['region'].lower() == 'russia':
            out = 'ad'
        else:
            out = 'other'
    return out

log['source_type'] = log.apply(source_type, axis=1)
log.head() # результат

# log[(log['region'].str.lower() == 'russia') & (log['traffic_source'].str.lower() == 'paid')].head() # проверка результатов
# log[log['region'].str.lower() == 'russia'] # тренировка с lower
# log[(log['traffic_source'] == 'yandex') | (log['traffic_source'] == 'direct')].head() # тренировка с фильтрами

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


In [3]:
# Задание 2
# В файле URLs.txt содержатся URL страниц новостного сайта. Вам нужно отфильтровать его по адресам страниц с текстами новостей.
# Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис.
# Выполните действия:
    # Прочитайте содержимое файла с датафрейм.
    # Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствие с заданным шаблоном.

url = pd.read_csv('./hw_4/URLs.txt') # гружу как csv, честно говоря, сомневался, но загрузилось корректно
url[url['url'].str.contains(r'/\d{8}-')].head() # вывожу результат. как-то слишком просто

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [4]:
# Задание 3
# Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. 
# Под временем жизни понимается разница между максимальным и минимальным значениями столбца timestamp для данного значения userId.

ml = pd.read_csv('./hw_4/ratings.csv') # загружаю данные
mlpivot = ml.pivot_table(index='userId', values='timestamp', aggfunc={'timestamp': [np.count_nonzero, min, max]},).reset_index() # pivot with all necessary fields
mlpivot = mlpivot[mlpivot['count_nonzero'] > 100].sort_values(by='count_nonzero', ascending=False) # filter 100+, sort desc
mlpivot['diff'] = mlpivot.apply(lambda row: row['max'] - row['min'], axis=1) # calculate lifetime as diff between max and min
mlpivot['diff'].mean() # show result
# mlpivot.to_excel('./hw_4/out.xlsx') # выгрузка в excel для проверки

40080507.4496124

In [5]:
# Задание 4
# Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе «Материалы для лекции “Продвинутый pandas”»
# Ноутбуки к лекции «Продвинутый pandas»).Нужно сформировать две таблицы:

# таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;
# аналогичную таблицу по типам выручки с указанием адреса клиента.
# Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

# Вводные данные для ДЗ
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

# Решение
client_base_full = functools.reduce(lambda a, b: a.merge(b, how='left', on='client_id'), [client_base, rzd, auto, air]) # полная таблица с адресом
client_base_short = client_base_full.drop(columns=['address']) # сокращённая таблица без

In [6]:
# Результат без адреса
client_base_short

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [7]:
# результат с адресом
client_base_full

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0
